In [ ]:
import numpy as np 
from scipy.stats import norm

def posterior_pdf_motion_model_odom(
    x_t0, x_t1, u_t1, alpha=[np.sqrt(0.05),np.sqrt(0.05),np.sqrt(0.05),np.sqrt(0.05)]):
    # Implements the PDF value for the posterior prob p(x_t1 | u_t1, x_t0)
    # x = [x, y, theta]
    # u = [drot1, drot2, dtrans]
    # The PDF is derived from a normal distribution

    [drot1, drot2, dtrans] = u_t1
    print(f"Odometry (drot1, drot2, dtrans): {u_t1}")

    # Odometry derived from the hypothesis
    [drot1_h, drot2_h, dtrans_h] = compute_odom_u_from_poses(x_t0, x_t1)
    print(f"Hypothesis (drot1, drot2, dtrans): {[drot1_h, drot2_h, dtrans_h]}")

    [a1, a2, a3, a4] = alpha
    p1 = norm.pdf(drot1 - drot1_h, scale=a1*np.abs(drot1_h) + a2*np.abs(dtrans_h))
    p2 = norm.pdf(dtrans - dtrans_h, scale=a3*np.abs(dtrans_h) + a4*(np.abs(drot1_h)+np.abs(drot2_h)))
    p3 = norm.pdf(drot2 - drot2_h, scale=a1*np.abs(drot2_h) + a2*np.abs(dtrans_h))
    return [p1, p2, p3, p1 * p2 * p3]

def compute_odom_u_from_poses(x_start, x_final):
    # Input poses x = [x, y, theta]
    [x0, y0, theta0] = x_start
    [x1, y1, theta1] = x_final
    dtrans = np.sqrt(((y1-y0)**2) + ((x1-x0)**2))
    drot1 = (np.arctan2(y1-y0, x1-x0) - theta0)
    drot2 = theta1 - theta0 - drot1
    return [drot1, drot2, dtrans]


# Initial, final-hypothesis, and final-odometry poses
x0 = [0, 0, 0]
x1 = [1.8, 0.75, np.pi/4.0]
x1_odom = [1.85, 0.77, np.pi/3.9]
u1 = compute_odom_u_from_poses(x0, x1_odom)
p = posterior_pdf_motion_model_odom(x0, x1, u1)
print(f"[p1, p2, p3, p1*p2*p3] = {p}")